In [1]:
import os
import copy
import dota2api
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB

%matplotlib inline

In [2]:
API_KEY = '2029A39ECAAB627B5F71CF4C3E30F72D'
api = dota2api.Initialise(API_KEY)

In [3]:
data = pd.read_csv('../data/draft/capmodedata.csv')

In [29]:
data.head(3)

,match_id,radiant_win,duration,1is_pick,1team_id,1hero_id,2is_pick,2team_id,2hero_id,3is_pick,...,17hero_id,18is_pick,18team_id,18hero_id,19is_pick,19team_id,19hero_id,20is_pick,20team_id,20hero_id
0,1928898739,True,2885,0,0,73,0,1,69,0,...,41.0,0,0.0,81.0,1,1.0,105.0,1,0.0,94.0
1,1928932285,True,1894,0,0,69,0,1,102,0,...,40.0,0,0.0,28.0,1,1.0,97.0,1,0.0,62.0
2,1928903165,True,2424,0,1,85,0,0,68,0,...,30.0,0,1.0,47.0,1,0.0,58.0,1,1.0,33.0


In [5]:
# # # Converting boolean value into number
boolean_values = 20
for i in range(boolean_values):
    pick_id = '{}is_pick'.format(i+1)
    data[pick_id] = data[pick_id].apply(lambda x: 1 if x else 0)

In [6]:
data.head(3)

,match_id,radiant_win,duration,1is_pick,1team_id,1hero_id,2is_pick,2team_id,2hero_id,3is_pick,...,17hero_id,18is_pick,18team_id,18hero_id,19is_pick,19team_id,19hero_id,20is_pick,20team_id,20hero_id
0,1928898739,True,2885,0,0,73,0,1,69,0,...,41.0,0,0.0,81.0,1,1.0,105.0,1,0.0,94.0
1,1928932285,True,1894,0,0,69,0,1,102,0,...,40.0,0,0.0,28.0,1,1.0,97.0,1,0.0,62.0
2,1928903165,True,2424,0,1,85,0,0,68,0,...,30.0,0,1.0,47.0,1,0.0,58.0,1,1.0,33.0


In [ ]:
partial_data = data.iloc[:, 3:18]

In [ ]:
partial_data.head(2)

In [ ]:
hero_6 = data['6hero_id']

In [ ]:
X = partial_data.as_matrix()
print(X.shape)
print(type(X))

In [ ]:
y = np.array(hero_6)
print(y.shape)
print(type(y))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model4 = MLPClassifier()

In [ ]:
model4.fit(X_train, y_train)

In [ ]:
y_pred = model4.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
np.unique(y_pred)

In [ ]:
len(np.unique(y_test))

In [ ]:
5.,  7., 17., 26., 29., 30., 35., 39., 50., 51., 57., 70., 75.,
       85., 99.

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, y_pred)

In [7]:
hero_1 = data['1hero_id']
heropick_1 = data['1is_pick']
hero_2 = data['2hero_id']
heropick_2 = data['2is_pick']
hero_3 = data['3hero_id']
heropick_3 = data['3is_pick']
hero_4 = data['4hero_id']
heropick_4 = data['4is_pick']
hero_5 = data['5hero_id']
heropick_5 = data['5is_pick']
hero_6 = data['6hero_id']
heropick_6 = data['6is_pick']

hero_7 = data['7hero_id']
# hero_8 = data['8hero_id']
# hero_9 = data['9hero_id']

In [8]:
test1 = pd.DataFrame(columns=['1hero_id','1is_pick' ,'2hero_id','2is_pick', '3hero_id','3is_pick', '4hero_id', '4is_pick', '5hero_id', '5is_pick','6hero_id', '6is_pick'])

In [9]:
# # Human = 0
# # Machine = 1

In [10]:
test1['1hero_id'] = hero_1
test1['1is_pick'] = heropick_1
test1['2hero_id'] = hero_2
test1['2is_pick'] = heropick_2
test1['3hero_id'] = hero_3
test1['3is_pick'] = heropick_3
test1['4hero_id'] = hero_4
test1['4is_pick'] = heropick_4
test1['5hero_id'] = hero_5
test1['5is_pick'] = heropick_5
test1['6hero_id'] = hero_6
test1['6is_pick'] = heropick_6
# test1['7hero_id'] = hero_7
# test1['8hero_id'] = hero_8
# test1['9hero_id'] = hero_9

In [11]:
test1.head(3)

,1hero_id,1is_pick,2hero_id,2is_pick,3hero_id,3is_pick,4hero_id,4is_pick,5hero_id,5is_pick,6hero_id,6is_pick
0,73,0,69,0,71,0,7,0,28,1,68.0,1
1,69,0,102,0,32,0,61,0,74,1,112.0,1
2,85,0,68,0,75,0,2,0,73,1,50.0,1


In [12]:
X = test1.as_matrix()
X

/home/sulabh/software/anaconda3/envs/dotaenv36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[ 73.,   0.,  69., ...,   1.,  68.,   1.],
       [ 69.,   0., 102., ...,   1., 112.,   1.],
       [ 85.,   0.,  68., ...,   1.,  50.,   1.],
       ...,
       [ 44.,   0.,  29., ...,   1.,  18.,   1.],
       [  1.,   0.,  57., ...,   1.,  28.,   1.],
       [ 71.,   0., 100., ...,   1.,  11.,   1.]])

In [13]:
X.shape

(461050, 12)

In [14]:
y = np.array(hero_7)

In [15]:
y

array([85., 31., 57., ..., 98.,  7., 26.])

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
np.unique(y_pred)

In [ ]:
# np.unique(y_test)

In [ ]:
model2 = MultinomialNB()

In [ ]:
model2.fit(X_train, y_train)

In [ ]:
y_pred = model2.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
# # MLP

In [19]:
from sklearn.neural_network import MLPClassifier

In [20]:
model3 = MLPClassifier()

In [21]:
model3.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [22]:
print('Trainig is Finished')

Trainig is Finished


In [23]:
y_pred = model3.predict(X_test)

In [25]:
from sklearn.metrics import accuracy_score

In [26]:
accuracy_score(y_test, y_pred)

0.06940678885153453

In [27]:
np.unique(y_pred)

array([  5.,   7.,   8.,  11.,  12.,  16.,  17.,  21.,  22.,  26.,  29.,
        30.,  32.,  36.,  39.,  42.,  48.,  50.,  51.,  54.,  55.,  57.,
        62.,  71.,  72.,  75.,  83.,  84.,  85.,  91., 100., 101.])

In [28]:
np.unique(y_test)

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,
        35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,
        46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,
        57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,
        68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,
        79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,
        90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100.,
       101., 102., 103., 104., 105., 106., 109., 110., 112.])